In [ ]:
# ============ Base imports ======================
import os
# ====== External package imports ================
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# ====== Internal package imports ================
from src.main.precision_recall import Analysis
# ============== Logging  ========================
import logging
from src.modules.utils.setup import setup, IndentLogger
logger = IndentLogger(logging.getLogger(''), {})
# =========== Config File Loading ================
from src.modules.utils.config_loader import get_config
conf  = get_config()
# ================================================

In [ ]:
setup('pr_curves')
vid_ref = ''
analysis_group = Analysis(model=40, vid_ref=vid_ref)

In [3]:
import cv2
import subprocess as sp
import shlex
from functools import partial

In [ ]:
def show_img(image):
    fig, ax = plt.subplots(figsize=(16, 9))
    ax.axis('off')
    plt.imshow(image)

In [ ]:
box_color_true = (0,0,255)
box_color_pred_tp = (0,255,0)
box_color_pred_fp = (255,0,0)
font_scale = .5
for i, frame in enumerate(frame_list):
    boxes = analysis_group.true_boxes[analysis_group.true_boxes['frame'] == i+1][['xtl','ytl','xbr','ybr']].values
    labels = analysis_group.true_boxes[analysis_group.true_boxes['frame'] == i+1][['label']].values
    for box, label in zip(boxes, labels):
        cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), box_color_true, 2)
        cv2.putText(frame, label[0], (int(box[0]), int(box[3])), cv2.FONT_HERSHEY_COMPLEX_SMALL, font_scale, box_color_true)
        
    relevant_pred = analysis_group.pred_boxes[(analysis_group.pred_boxes['frame'] == i+1) & 
                                              (analysis_group.pred_boxes['confidence']>.1)]
    boxes = relevant_pred[['xtl','ytl','xbr','ybr']].values
    labels = relevant_pred[['label']].values
    true_pos = relevant_pred[['true_positive']].values
    for box, label, true_positive in zip(boxes, labels, true_pos):
        if true_positive:
            cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), box_color_pred_tp, 2)
            cv2.putText(frame, label[0], (int(box[0]), int(box[3])), cv2.FONT_HERSHEY_COMPLEX_SMALL, font_scale, box_color_pred_tp)
        else:
            cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), box_color_pred_fp, 2)
            cv2.putText(frame, label[0], (int(box[0]), int(box[3])), cv2.FONT_HERSHEY_COMPLEX_SMALL, font_scale, box_color_pred_fp)
        
    
    show_img(frame)
    #plt.savefig(f'frame_{i}.png')
    plt.show()
    plt.close()